In [477]:
import os
import functools
import sys
import numpy as np
import pandas as pd
import random
import re
import matplotlib.pyplot as plt
import tqdm
import nltk
from sklearn.model_selection import train_test_split
from collections import Counter
import io, sys, math, re
from collections import defaultdict
from sklearn.metrics import accuracy_score
import csv

from nltk.corpus import stopwords
nltk.download('stopwords')

import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/lyuxiaoxi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# load raw data

In [394]:
def load_imdb(base_csv, test_size):
    """
    Load the IMDB dataset
    :param base_csv: the path of the dataset file.
    :return: train and test set.
    """
    df = pd.read_csv(base_csv, engine='python')
    df.head()
    reviews, sentiments = df['review'].values, df['sentiment'].values
    
    # we are adding an end of sentence word(eos) at the end of the sentence to help
    #    us know when the sentence ends when generating a sentence
    reviews=[i+" eos" for i in reviews]
    
    # remove useless characters '<br />'
    for i, element in enumerate(reviews):
        element = element.replace('<br />',' ')
        reviews[i] = element
    
    return reviews, sentiments

In [395]:
base_csv='IMDBDataset.csv'
test_size=0.2
reviews, sentiments = load_imdb(base_csv, test_size)

# Split data
x_train, x_test, y_train, y_test = train_test_split(reviews, sentiments, test_size=test_size, stratify=sentiments, random_state=233)
print(f'length of train data is {len(x_train)}')
print(f'length of test data is {len(x_test)}')

length of train data is 40000
length of test data is 10000


In [396]:
print(x_train[0])

Man, I really love the new DVD that Universal put out. I've never seen THE SENTINEL look this good since I had to put up with crappy, grainy VHS tapes for years. Unfortunately there are no extras beyond a trailer that looks pretty worse for wear. And AVOID the Goodtimes DVD at all costs. It sucks.  Anyway, troubled fashion model Alison Parker (Cristina Raines) moves into haunted NYC brownstone, only it's more than just haunted. It's also a portal to hell and the Vatican keeps an old blind priest (John Carradine) to keep watch over it and make sure the devils and arch-angels don't escape.   This has an all star cast full of old-timey actors like Ava Gardner, Arthur Kennedy, Jose Ferrer etc... as well as cameos of upcoming 80s stars including Christopher Walken, Jeff Goldblum (who's voice was mysteriously overdubbed) and Tom Berenger. And you won't even recognize Jerry Orbach from LAW & ORDER. I had to do a double-take when I didn't quite place where I'd seen him before.  Nice gore scene

In [398]:
print(y_train[0])

positive


##  Step 2: 
### Identify or construct a solution based on a generative probabilistic (language) model. Describe the model in detail and develop a solution using parameter inference (and/or decoding).

### Method 1: N-gram for sentiment analysis

Codes reference link: 
- https://github.com/siddiquiamir/NLTK-Text-Mining
- https://www.youtube.com/watch?v=ic5XcUxiUrI

N-gram concepts (for report writing):
- https://www.cnblogs.com/bep-feijin/p/9430164.html

CountVectorize: 
- https://blog.csdn.net/weixin_38278334/article/details/82320307
- https://blog.csdn.net/appleyuchi/article/details/108261497
- https://www.biaodianfu.com/sklearn-feature-extraction-text.html

#### Convert the dataset into a matrix 
p.s. parameters in function CountVectorizer will be tuned in later steps, such as ngram_range...

In [483]:
"""
Try different N in N-gram model
"""

x_train_vectorized = []
vect = []

vect_1 = CountVectorizer(min_df=5, ngram_range=(1,1)).fit(x_train)
x_train_vectorized_1 = vect_1.transform(x_train)
x_train_vectorized.append(x_train_vectorized_1)
vect.append(vect_1)

vect_2 = CountVectorizer(min_df=5, ngram_range=(2,2)).fit(x_train)
x_train_vectorized_2 = vect_2.transform(x_train)
x_train_vectorized.append(x_train_vectorized_2)
vect.append(vect_2)

vect_3 = CountVectorizer(min_df=5, ngram_range=(3,3)).fit(x_train)
x_train_vectorized_3 = vect_3.transform(x_train)
x_train_vectorized.append(x_train_vectorized_3)
vect.append(vect_3)

vect_4 = CountVectorizer(min_df=5, ngram_range=(4,4)).fit(x_train)
x_train_vectorized_4 = vect_4.transform(x_train)
x_train_vectorized.append(x_train_vectorized_4)
vect.append(vect_4)

vect_5 = CountVectorizer(min_df=5, ngram_range=(5,5)).fit(x_train)
x_train_vectorized_5 = vect_5.transform(x_train)
x_train_vectorized.append(x_train_vectorized_5)
vect.append(vect_5)

#### Train a logistic model with different N (from 1 to 5).

In [485]:
prediction_lis = []
model = []
for i in range(5):
    m = LogisticRegression()
    m.fit(x_train_vectorized[i], y_train)
    predictions = m.predict(vect[i].transform(x_test))
    prediction_lis.append(predictions)
    model.append(m)

/Users/lyuxiaoxi/miniforge3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/lyuxiaoxi/miniforge3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#log

#### Model evaluation with different N (from 1 to 5).

In [487]:
auc_lis = []
for j in range(5):
    for i in range(len(prediction_lis[j])):
        if prediction_lis[j][i]=='positive':
            prediction_lis[j][i]=1
        elif prediction_lis[j][i]=='negative':
            prediction_lis[j][i]=0
    for i in y_test:
        if i=='positive':
            i=1
        elif i=='negative':
            i=0      
    auc = roc_auc_score(y_test, prediction_lis[j])
    print("AUC score is", auc)
    auc_lis.append(auc)
    # feature_names = np.array(vect[j].get_feature_names())
    # sorted_coef_index = model[j].coef_[0].argsort()
    # print("Negative words", feature_names[sorted_coef_index[:10]])
    # print("Positive words", feature_names[sorted_coef_index[:-11:-1]])

AUC score is 0.8867999999999999
AUC score is 0.8876
AUC score is 0.8503
AUC score is 0.7843
AUC score is 0.6868


### -> We find that when N=2 (i.e. bigram), prediction accuracy using Logistic Regression is the highest; when N=1 or 2 or 3, prediction accuracy are all pretty high (>85%).

## Step 5: Train and apply both approaches to synthetic data that you generate according to the generative model from Step 2. Evaluate the results qualitatively and quantitatively. Highlight situations where each approach performs well and poorly.

## Method : N-gram

In [343]:
"""
Data preparation
"""

df = pd.read_csv('IMDBDataset.csv', engine='python')
df_pos = df[df['sentiment'] =='positive']
df_neg = df[df['sentiment'] =='negative']
x_pos, y_pos = df_pos['review'].values, df_pos['sentiment'].values
# we are adding an end of sentence word(eos) at the end of the sentence to help
#    us know when the sentence ends when generating a sentence
x_pos = [i+" eos" for i in x_pos]
# remove useless characters 
for i, element in enumerate(x_pos):
    element = element.replace('<br />',' ')
    element = element.replace('.',' ')
    element = element.replace(',',' ')
    element = element.replace('!',' ')
    element = element.replace('...',' ')
    element = element.replace('(',' ')
    element = element.replace(')',' ')
    element = element.replace(':',' ')                    
    x_pos[i] = element
x_neg, y_neg = df_neg['review'].values, df_neg['sentiment'].values
x_neg = [i+" eos" for i in x_neg]
for i, element in enumerate(x_neg):
    element = element.replace('<br />',' ')
    element = element.replace('.',' ')
    element = element.replace(',',' ')
    element = element.replace('!',' ')
    element = element.replace('...',' ')
    element = element.replace('(',' ')
    element = element.replace(')',' ')
    element = element.replace(':',' ')
    x_neg[i] = element

#### Implement the N-Gram Text Generator model 
Q: How does the N-Gram Text Generator model work?

A: The logic behind the model is the following – given two words, we check for Tri-Grams that start with these words and the algorithm picks one based on its probability. 

For example, let’s say we have the words “if” and “you”, and there are the following trigrams that start with them along with their counts “if you agree”:25 and “if you believe”:5. The algorithm will pick one of these two based on the probabilities 25/30 for the first one and 5/30 for the second one. This will continue until we have the word “eos” as our next word. 

### To deal with rare n-grams (i.e., the n-gram is not in the original dataset), generally we have 3 options - smoothing, backoff, and interpolation. We will try some of them as follows. 
### 1. Kneser Ney Smoothing
reference list:
- https://stackoverflow.com/questions/43939344/compute-ngrams-across-a-list-of-lists-of-sentences-using-nltk
- https://stackoverflow.com/questions/61761215/how-to-perform-kneser-ney-smoothing-in-nltk-at-word-level-for-bigram-language-mo
- https://tedboy.github.io/nlps/generated/generated/nltk.KneserNeyProbDist.html
- https://www.nltk.org/api/nltk.probability.CrossValidationProbDist.html?highlight=probability+probdist#nltk.probability.CrossValidationProbDist.samples
- https://predictivehacks.com/text-generation-for-instagram-using-n-grams/
- https://practicaldatascience.co.uk/machine-learning/how-to-use-count-vectorization-for-n-gram-analysis

In [345]:
"""
We choose 3-gram in this case, since n must be 3 in function nltk.KneserNeyProbDist().
"""

n=3 
freq_dist_pos = nltk.FreqDist() 
for sent in x_pos:
    freq_dist_pos.update(nltk.ngrams(sent.split(), n))
kneser_ney_pos = nltk.KneserNeyProbDist(freq_dist_pos)
freq_dist_neg = nltk.FreqDist() 
for sent in x_neg:
    freq_dist_neg.update(nltk.ngrams(sent.split(), n))
kneser_ney_neg = nltk.KneserNeyProbDist(freq_dist_neg)

In [351]:
"""
Generate synthetic data (i.e., 20 new reviews) according to the generative model in Step 2.
"""

def word_gen(start_word, kneser_ney):
    # initialize list_words as start_word
    list_words = []
    list_words.append(start_word[0])
    list_words.append(start_word[1])
    # we make constraint that length of each generated review should be limited within 200 words
    while (list_words[-1]!='eos' and len(list_words)<=100): 
        # stochastic: randomly choose a word
        words_freq = dict()
        for i in kneser_ney.samples():
            if (i[0]==list_words[-2] and i[1]==list_words[-1]):
                words_freq[i] = kneser_ney.prob(i) 
        # if the gram is NOT in kneser_ney, add the start word into this sentence.
        if (len(words_freq)==0):
            for i in kneser_ney.samples():
                if (i[0]==start_word[0] and i[1]==start_word[1]):
                    words_freq[i] = kneser_ney.prob(i) 
        # random select a gram, according to the freq of each gram
        word = random.choices(list(words_freq.keys()), weights=words_freq.values(), k=1)
        list_words.append(word[0][-2])  
        list_words.append(word[0][-1])  
    # generate a sentence
    generated_sentence = ' '.join(list_words[:-1])
    return generated_sentence

def generate_synthetic_datasets(freq_dist, kneser_ney):
    freq_dist_most_common = freq_dist.most_common(10) # generate 20 new reviews
    lis_generated_sentences = []
    for i in freq_dist_most_common:
        start_word = i[0]
        start_word = start_word[0:2]
        generated_sentence = word_gen(start_word, kneser_ney)
        lis_generated_sentences.append(generated_sentence)
    return lis_generated_sentences

lis_generated_sentence_pos = generate_synthetic_datasets(freq_dist_pos, kneser_ney_pos)
generated_review_pos = lis_generated_sentence_pos
generated_sentiment_pos = np.linspace(1,1,10)
lis_generated_sentence_neg = generate_synthetic_datasets(freq_dist_neg, kneser_ney_neg)
generated_review_neg = lis_generated_sentence_neg
generated_sentiment_neg = np.linspace(0,0,10)

In [352]:
"""
Train the N-Gram Model we builtup in Step 2 on the synthetic data.
"""

generated_review = generated_review_pos + generated_review_neg
generated_sentiment = [*generated_sentiment_pos, *generated_sentiment_neg] 
generated_predictions = model.predict(vect.transform(generated_review))
for i in range(len(generated_predictions)):
    if generated_predictions[i]=='positive':
        generated_predictions[i]=1
    elif generated_predictions[i]=='negative':
        generated_predictions[i]=0
print("AUC score is", roc_auc_score(generated_sentiment, generated_predictions))

AUC score is 0.5


### -> Accuracy of smoothing is very low, so we try Backoff as follows.

### 2. Backoff (with N=3)

reference list:
- https://github.com/AmrKhalifa/ML_DL-Tutorials/blob/master/NLP_Lab_sessions/lab2:%20N-gram%20Language%20Modeling/(solution).ipynb- 
- https://www.youtube.com/watch?v=gHC9tRyVSNE

In [384]:
lis_x_pos = []
for sent in x_pos:
    l = list(sent.split())
    lis_x_pos.append(l)
lis_x_neg = []
for sent in x_neg:
    l = list(sent.split())
    lis_x_neg.append(l)

In [431]:
def build_ngram(data, n):
    '''
    Parameters:
    data (list of element): each element is a sentence of the text 
    n (int): size of the n-gram
    
    Returns:
    proba (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    }
    '''
    total_number_words = 0
    counts = defaultdict(lambda: defaultdict(lambda: 0.0))

    for sentence in data:
        sentence = tuple(sentence)
        for i in range(len(sentence)):
            total_number_words +=1
            for k in range(n):
                if i-k < 0:
                    break
                counts[sentence[i-k:i]][sentence[i]] +=1 

    proba  = defaultdict(lambda: defaultdict(lambda: 0.0))
    for context in counts.keys():
        denom =0
        for w in counts[context].keys():
            denom += counts[context][w]
        for w in counts[context].keys():
            proba[context][w] = counts[context][w]/denom 
    
    return proba


def get_proba_distrib(model, context):
    '''
    Parameters: 
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    }
    context (list of strings): the sentence we need to find the words after it and 
    thier probabilites
    
    Returns:
    words_and_probs(dic): {word: probability of word given context}
    
    '''
    # find the longest available ngram
    if context in model:
        return model[context]
    else:
        return get_proba_distrib(model, context[1:])
    

def generate(model):
    '''
    Parameters: 
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    }
    
    Returns:
    sentence (list of strings): a sentence sampled according to the language model. 
    '''
    sentence = [""]
    while sentence[-1] != "eos" and len(sentence)<100:
        proba = get_proba_distrib(model, tuple(sentence))
        w = np.random.choice((list(proba.keys())), 1, p = list(proba.values()))
        sentence.append(w[0])
    return sentence

In [507]:
"""
Train N-gram model for positive and negative reviews, respectively. 
We choose n=3 in this case, since prediction accuracy is the highest when n=2.
"""

n = 3
print("Build ngram model with n = ", n)
model_pos = build_ngram(lis_x_pos, n)
model_neg = build_ngram(lis_x_neg, n)

Build ngram model with n =  3


In [508]:
"""
Generate synthetic data (i.e., 500 new reviews for positive and negative respectively) according to the generative model in Step 2.
"""

generated_review_pos = []
generated_review_neg = []
for i in range(500):
    generated_review_pos.append(generate(model_pos))
    generated_review_neg.append(generate(model_neg))   
generated_sentiment_pos = np.linspace(1,1,500)
generated_sentiment_neg = np.linspace(0,0,500)

In [509]:
"""
Train the N-Gram Model we builtup in Step 2 on the synthetic data.
"""

generated_review = generated_review_pos + generated_review_neg
generated_sentiment = [*generated_sentiment_pos, *generated_sentiment_neg] 

lis_generated_review = []
for r in generated_review:
    r_str = ' '.join(r) 
    lis_generated_review.append(r_str)
generated_review = lis_generated_review 
generated_predictions = model[2].predict(vect[2].transform(generated_review))

for i in range(len(generated_predictions)):
    if generated_predictions[i]=='positive':
        generated_predictions[i]=1
    elif generated_predictions[i]=='negative':
        generated_predictions[i]=0
print("AUC score is", roc_auc_score(generated_sentiment, generated_predictions))

AUC score is 0.8290000000000001


In [510]:
generated_sentiment = np.array(generated_sentiment)
generated_sentiment = (np.rint(generated_sentiment)).astype(int)
generated_predictions = generated_predictions.astype(str).astype(int)
accuracy_score(generated_sentiment, generated_predictions)

0.829

### -> Accuracy of backoff is pretty high. 

In [511]:
"""
Save the generated sentences as an excel.
The first 500 sentences are positive, the last 500 sentences are negative. 
"""

generated_review_df = pd.DataFrame(generated_review, columns=["generated reviews"])
generated_review_df.to_csv('ngram_synthetic_data_1000.csv', index=False)